# Recon8D Example 
##### Author: Ryan Schildcrout

## Summary
##### This notebook generates the machine learning models used in Schildcrout, R., Smith, K., Bhowmick, R., Lu, Y., Menon, S., Kapadia, M., Kurtz, E., Coffeen-Vandeven, A., Nelakuditi, S., & Chandrasekaran, S. (2025). Recon8D: A metabolic regulome network from oct-omics and machine learning (p. 2024.08.17.608400). bioRxiv. https://doi.org/10.1101/2024.08.17.608400


## 1. Load in necessary packages and data
##### This notebook was written to work in a conda environment. All required packages to generate Recon8D models are listed below. 

In [5]:
import sys
import pandas as pd
import time
import sklearn
import numpy as np
from collections import Counter
import scipy
import dill

# Machine learning
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from xgboost import XGBRegressor

##### Sample data (using histone PTMs to predict metabolome variation) can be found in the example_datasets folder

In [ ]:
# Read in data (rows = cell lines, columns = features)
input_features = pd.read_csv('./your_file_here.csv',index_col=0)
metabolomics = pd.read_csv('./your_metabolomics_here.csv',index_col=0)

## 2. Data preprocessing
##### A. Remove features with low variance

In [ ]:
# Function for removing features with low variance
def remove_lowVar(df, minVariance):
  selector = VarianceThreshold(threshold = minVariance) 
  selector.fit(df)
  df_new = df.loc[:, selector.get_support()]
  print("Removed {} of {} metabolites for having zero variance".format(df.shape[1]- df_new.shape[1], df.shape[1]),flush=True)
  return(df_new)

In [ ]:
# Call function to remove features with zero variance
print("Your_feature_type_here...")
feature_highVar = remove_lowVar(input_features, 0)

print("Metabolites...")
met_highVar = remove_lowVar(metabolomics, 0)

##### B. Make lists of feature names and metabolites

In [ ]:
feature_names = feature_highVar.columns
metabolite_names = met_highVar.columns

##### C. Merge datasets by cell line

In [ ]:
print("\nFEATURE:")
print(feature_highVar.shape)
print(met_highVar.shape)
merged_data =  pd.merge(feature_highVar, met_highVar, left_index=True, right_index=True)
print(merged_data.shape)

## 3. Machine Learning

##### A. Function for RandomForests and XGBoost

In [ ]:
# Set scaler (for z-scoring)
scaler = StandardScaler()
def train_multiRegressCV(classifier, data, x_cols, y_cols, n_splits=5, pval_cutoff=0.05, scale_y=True):

  # Identify features and metabolites from input lists
  X =  data.loc[:, x_cols]
  y =  data.loc[:, y_cols]

  # Set the classifier for the type of algorithm entered
  if classifier=="RF":
    model=RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1, max_depth=10)
    print("Training model with random forest!")
  elif classifier=="XGBoost":
    model=XGBRegressor(random_state=0,n_estimators=100,n_jobs=-1)
    print("Training model with XGBoost")
  else:
    print("Enter model type!")
    sys.exit()

  # Create multioutput model (one model trained for each metabolite)
  clf = MultiOutputRegressor(model)

  # Create cross validation splits
  cv = KFold(n_splits=n_splits,
                shuffle=True,
                random_state=123)

  # Create empty lists to store test and prediction data
  df_y_test_all = pd.DataFrame(columns=y_cols)
  df_y_pred_all = pd.DataFrame(columns=y_cols)

  count = 1

  # loop through cross-val folds
  for train_index, test_index in cv.split(X, y):
      
      # Start timer
      start_time = time.perf_counter()

      print("CV fold:")
      print(count,flush=True)

      # Set train and test data for CV splits
      X_trainCV, X_testCV = X.iloc[train_index], X.iloc[test_index]
      y_trainCV, y_testCV = y.iloc[train_index], y.iloc[test_index]

      # Scale data (z-score) based on training set
      X_trainCV =  scaler.fit_transform(X_trainCV)
      X_testCV =  scaler.transform(X_testCV)

      if scale_y:
        y_trainCV =  scaler.fit_transform(y_trainCV)
        y_testCV =  scaler.transform(y_testCV)

      # Train ML model!
      tmp_mdl = clf.fit(X_trainCV, y_trainCV)
      y_predCV = tmp_mdl.predict(X_testCV)

      df_pred_temp  = pd.DataFrame(y_predCV, columns=y_cols)
      df_test_temp = pd.DataFrame(y_testCV, columns=y_cols)

      # Concatenate data from each CV
      df_y_test_all = pd.concat([df_y_test_all, df_test_temp])
      df_y_pred_all = pd.concat([df_y_pred_all, df_pred_temp])

      count = count+1

      finish_time = time.perf_counter()
      print("CV fold finished in {} seconds".format(finish_time-start_time),flush=True)


  print("Calculating Pearson's R for each metabolite...")

  # Calculate Pearson's R for predicted vs test data
  r = []
  for i, col in enumerate(y_cols):
    r.append(scipy.stats.pearsonr(df_y_test_all.iloc[:,i], df_y_pred_all.iloc[:,i]))

  df_results = pd.DataFrame(r, columns=['R','pval'], index=y_cols)

  # Determine significance based on FDR-corrected P value
  # R must be positive, as we're looking at true vs predicted values
  df_results['Significant']  = (df_results.pval < pval_cutoff) & (df_results.R > 0)
  df_results['R2']  = df_results.R**2

  # Scale all data
  X_final = scaler.fit_transform(X)
  X_final = pd.DataFrame(X_final,columns=X.columns,index=X.index)
  if scale_y:
    y_final = scaler.fit_transform(y)
    y_final = pd.DataFrame(y_final,columns=y.columns,index=y.index)

  # Train final model on all data
  print("Training final model on dataset of {} samples and {} features".format(X_final.shape[0], X_final.shape[1]))
  final_mdl = clf.fit(X_final, y_final)

  # Output: dataframe with results, multi-output model containing models for each metabolite
  return df_results, final_mdl

##### B. Function for Ridge and Lasso regression

In [ ]:
# Set scaler (for z-scoring)
scaler = StandardScaler()
def train_tune_CV(classifier,data, x_cols, y_cols,
                  n_splits=5, pval_cutoff=0.05, alphas=[1e-2, 0.1, 1, 10, 100, 1000], scale_y=True):

  # Identify features and metabolites from input lists
  X =  data.loc[:, x_cols]
  y =  data.loc[:, y_cols]

  # Set the classifier for the type of algorithm entered
  if classifier=="Ridge":
    model=Ridge(alpha=1.0, max_iter=None, tol=0.001, solver='auto', random_state=0)
    print("Training model with Ridge!")
  elif classifier=="Lasso":
    model=Lasso(alpha=1.0, max_iter=500, tol=0.001, random_state=0)
    print("Training model with Lasso!")
  else:
    print("Enter model type!")
    sys.exit()

  # Create multioutput model (one model trained for each metabolite)
  multi_ridge = MultiOutputRegressor(model)

  # Iterate through alphas
  hyperParameters = {'estimator__alpha':alphas}
  gridSearch = GridSearchCV(multi_ridge, hyperParameters, scoring='r2', cv=5)

  cv = KFold(n_splits=n_splits,
                shuffle=True,
                random_state=123)

  # Create empty lists to store test and prediction data
  df_y_test_all = pd.DataFrame(columns=y_cols)
  df_y_pred_all = pd.DataFrame(columns=y_cols)

  # Initialize dataframes for hyperparameters
  df_alpha_cv = pd.DataFrame()
  df_r_cv = pd.DataFrame()

  # Loop through cross-val folds
  count = 1

  for count, (train_index, test_index) in enumerate(cv.split(X, y)):
      print(count)
      X_trainCV, X_testCV = X.iloc[train_index], X.iloc[test_index]
      y_trainCV, y_testCV = y.iloc[train_index], y.iloc[test_index]

      # Scale data (z-score) based on training set
      X_trainCV =  scaler.fit_transform(X_trainCV)
      X_testCV =  scaler.transform(X_testCV)

      if scale_y:
        y_trainCV = scaler.fit_transform(y_trainCV)
        y_testCV = scaler.transform(y_testCV)

      # Train ML model!
      tmp_mdl = gridSearch.fit(X_trainCV, y_trainCV)

      # predict on CV-test set
      y_predCV = tmp_mdl.predict(X_testCV)

      # Store tuned alpha values
      my_alphas = []
      r_cv = []

      for i in range(len(tmp_mdl.best_estimator_.estimators_)):
        my_alphas.append(tmp_mdl.best_estimator_.estimators_[i].get_params()['alpha'])
        r_cv.append(scipy.stats.pearsonr(y_predCV[:,i], y_testCV[:,i])[0])

      df_alpha_cv.loc[:,count] = my_alphas
      df_r_cv.loc[:,count] = r_cv

      df_pred_temp  = pd.DataFrame(y_predCV, columns=y_cols)
      df_test_temp = pd.DataFrame(y_testCV, columns=y_cols)

      # Concatenate data from each CV
      df_y_test_all = pd.concat([df_y_test_all, df_test_temp])
      df_y_pred_all = pd.concat([df_y_pred_all, df_pred_temp])

  print("Calculating Pearson's R for each metabolite...")
  
  # Calculate Pearson's R for predicted vs test data
  r = []
  for i, col in enumerate(y_cols):
    r.append(scipy.stats.pearsonr(df_y_test_all.iloc[:,i], df_y_pred_all.iloc[:,i]))

  df_results = pd.DataFrame(r, columns=['R','pval'], index=y_cols)

  # Determine significance based on FDR-corrected P value
  # R must be positive, as we're looking at true vs predicted values
  df_results['Significant']  = (df_results.pval < pval_cutoff) & (df_results.R > 0)
  df_results['R2']  = df_results.R**2

  # Select best hyperparameters
  print("For each metabolite, find CV fold with best R and get alpha...")
  ix_best_fold = df_r_cv.idxmax(axis=1)
  best_alphas = df_alpha_cv.values[np.arange(len(ix_best_fold)),ix_best_fold]
  Counter(best_alphas)
  savetxt('best_alphas_phos.csv', best_alphas)

  print("Training final models with best alphas...")
  
  # Scale all data
  X_final = scaler.fit_transform(X)
  X_final = pd.DataFrame(X_final,columns=X.columns,index=X.index)
  if scale_y:
    y_final = scaler.fit_transform(y)
    y_final = pd.DataFrame(y_final,columns=y.columns,index=y.index)

  # Train final model on all data with proper alpha values
  final_mdls = list()
  for i in range(len(best_alphas)):
    if classifier=="Ridge":
        model=Ridge(alpha=best_alphas[i], max_iter=None, tol=0.001, solver='auto', random_state=0)
    else:
        model=Lasso(alpha=best_alphas[i], max_iter=500, tol=0.001, random_state=0)
    model.fit(X_final.iloc[:,:], y_final.iloc[:,i])
    final_mdls.append(model)

  # Output: dataframe with results, multi-output model containing models for each metabolite, and the best alpha values
  return df_results, final_mdls, best_alphas

##### C. P value correction and model training

In [ ]:
# Assign significance value (Bonferroni correction) for multiple hypothesis testing
pval_cutoff = 0.05 / len(metabolite_names)

In [ ]:
# Call ML function (either train_multiRegressCV for RF or XGB, or train_tune_CV for Ridge or Lasso)
results_df, model = train_multiRegressCV("RF",
                                            data = merged_data, # Whole merged data set
                                            x_cols = feature_names, # List of feature names
                                            y_cols = metabolite_names, # List of variable names
                                            pval_cutoff = pval_cutoff, # Significance value
                                            n_splits = 5) # Cross validation splits

print("Metabolites below Bonferroni pval cutoff: {}".format(np.sum(results_df.Significant)),flush=True)

In [ ]:
# Save model results
df = pd.DataFrame({"pearsons_r":results_df.R,
                                "model_pval":results_df.pval,
                                "metabolite_significant":results_df.Significant})
df.to_csv("./your_results.csv")

# Save models
print("Saving models!",flush=True)
with open('./your_final_model.pkl', 'wb') as f:
   dill.dump(model, f)
print("Done!",flush=True)

## 4. Generate feature importances from RF models

In [ ]:
# Load in RF models
print("Loading models!")
with open('./results/your_final_model.pkl', 'rb') as f:
   final_models = dill.load(f)
print("Done!")

# Pull first set of feature importances
importances = pd.DataFrame(final_models.estimators_[0].feature_importances_,index=feature_names)
# Append feature importances for all other metabolites
for i in range(0,224):
  importances = final_models.estimators_[i+1].feature_importances_
    # Convert to dataframe
  importances_df = pd.DataFrame(importances)
    # Label features
  importances_df = importances_df.set_index(feature_names)
    # Append features from next metabolite model
  importances[str(i+1)] = importances_df
  
importances = importances.set_axis(metabolite_names, axis=1)

## 5. Metabolome prediction using histone PTMs example

In [ ]:
# Set file paths and read in your feature and variable data
features_file_path = "./CCLE_hist.csv"
metabolomics_file_path = "./CCLE_metabolomics_averages.csv"
input_features = pd.read_csv(features_file_path, index_col=0)
metabolomics = pd.read_csv(metabolomics_file_path, index_col=0)

In [ ]:
# Call function to remove features with zero variance
print("Histone PTMs...")
feature_highVar = remove_lowVar(input_features, 0)

print("Metabolites...")
met_highVar = remove_lowVar(metabolomics, 0)

In [ ]:
# Get feature names
feature_names = input_features.columns
my_metabs = metabolomics.columns

In [ ]:
# Merge features+metabs by cell lines
print("\nFEATURE:")
print(feature_highVar.shape)
print(met_highVar.shape)
merged_data =  pd.merge(feature_highVar, met_highVar, left_index=True, right_index=True)
print(merged_data.shape)

In [ ]:
# Assign significance value (Bonferroni correction)
pval_cutoff_ccle = 0.05 / len(my_metabs)

In [ ]:
# Train ML models and save results
results_df, model = train_multiRegressCV("RF",
                                            data = merged_data,
                                            x_cols = feature_names,
                                            y_cols = my_metabs,
                                            pval_cutoff = pval_cutoff_ccle,
                                            n_splits = 5)

print("Metabolites below Bonferroni pval cutoff: {}".format(np.sum(results_df.Significant)),flush=True)

# Save results
df = pd.DataFrame({"pearsons_r":results_df.R,
                                "model_pval":results_df.pval,
                                "metabolite_significant":results_df.Significant})
df.to_csv("./CCLE_histone_results.csv")

# Save models
print("Saving models!",flush=True)
with open('./CCLE_histone_model.pkl', 'wb') as f:
   dill.dump(model, f)
print("Done!",flush=True)